In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob

In [2]:
# read in all casts processed in total_df.ipynb
total_df = pd.read_csv('../data/big_df.csv')

# display the first 5 rows of the df 
total_df.head()

,Pressure (dbar),Depth (m),Temperature (°C),Conductivity (µS/cm),Specific Conductance (µS/cm),Salinity (PSS),Sound Velocity (m/s),Density (kg/m³),Latitude,Longitude,UTC Time,File Name
0,0.15,0.153151,9.153524,296.339874,433.834991,0.204679,1444.146359,999.934361,40.274148,-106.8399,2025-05-13 18:12:11,CC2435009_20250513_181211
1,0.45,0.458792,9.255136,295.835466,431.811837,0.203774,1444.561198,999.927172,40.274148,-106.8399,2025-05-13 18:12:11,CC2435009_20250513_181211
2,0.75,0.764653,9.132318,295.647002,433.089545,0.204300,1444.069489,999.938574,40.274148,-106.8399,2025-05-13 18:12:11,CC2435009_20250513_181211
3,1.05,1.070511,9.033097,296.121842,435.049797,0.205173,1443.672646,999.948268,40.274148,-106.8399,2025-05-13 18:12:11,CC2435009_20250513_181211
4,1.35,1.376366,8.925369,296.570547,437.092585,0.206079,1443.240202,999.958485,40.274148,-106.8399,2025-05-13 18:12:11,CC2435009_20250513_181211


In [3]:
# Extract the first row for each cast (based on File Name)
cast_locations = total_df.groupby("File Name").first().reset_index()

# Optional: if you want only lat/lon and filename
cast_locations = cast_locations[["Latitude", "Longitude", "File Name"]]

In [4]:
cast_locations

,Latitude,Longitude,File Name
0,40.274148,-106.839900,CC2435009_20250513_181211
1,40.276507,-106.841752,CC2435009_20250513_181617
2,40.279905,-106.851733,CC2435009_20250513_182530
3,40.279532,-106.852019,CC2435009_20250513_182851
4,40.283628,-106.853450,CC2435009_20250513_194404
...,...,...,...
161,40.274410,-106.850507,CC2435009_20250519_182038
162,40.274005,-106.850038,CC2435009_20250519_182316
163,40.283275,-106.854001,CC2435009_20250519_185748
164,40.283448,-106.853752,CC2435009_20250519_190629


In [6]:
pip install folium


Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import folium


# Create map centered at mean lat/lon
m = folium.Map(location=[
    cast_locations["Latitude"].mean(),
    cast_locations["Longitude"].mean()
], zoom_start=13)

# Add cast locations as circle markers
for _, row in cast_locations.iterrows():
    folium.CircleMarker(
        location=[row["Latitude"], row["Longitude"]],
        radius=5,
        popup=row["File Name"],
        color="blue",
        fill=True,
        fill_color="blue"
    ).add_to(m)

# Save and/or display
m.save("stagecoach_casts_map.html")
m  # Will display inline in Jupyter


In [15]:
import pandas as pd
from shapely.geometry import Point, Polygon

# Define the polygons
polygons = {
    'DOC': Polygon([(40.28429, -106.85365), (40.28383, -106.85341), (40.28313, -106.85488), (40.28356, -106.85524)]),
    'HAR': Polygon([(40.28424, -106.85348), (40.28565, -106.85185), (40.28787, -106.85680), (40.28753, -106.85766)]),
    'MOR': Polygon([(40.27903, -106.84560), (40.28021, -106.84374), (40.27352, -106.83860), (40.27283, -106.84056)]),
    'GLZ': Polygon([(40.27291, -106.84515), (40.27610, -106.85043), (40.27509, -106.85397), (40.27129, -106.84970)]),
    'KEY': Polygon([(40.27805, -106.86361), (40.27730, -106.86697), (40.28124, -106.86839), (40.28180, -106.86597)]),
    'LIL': Polygon([(40.27265, -106.86163), (40.27315, -106.85961), (40.27214, -106.85906), (40.27175, -106.86068)]),
}

# Function to determine area
def get_area(lat, lon):
    point = Point(lat, lon)
    for name, poly in polygons.items():
        if poly.contains(point):
            return name
    return 'UNKNOWN'

# Apply function to DataFrame
cast_locations['Area'] = cast_locations.apply(lambda row: get_area(row['Latitude'], row['Longitude']), axis=1)


In [17]:
import folium
import matplotlib.pyplot as plt

# Choose distinct colors for each area
area_colors = {
    'DOC': 'red',
    'HAR': 'green',
    'MOR': 'purple',
    'GLZ': 'orange',
    'KEY': 'darkblue',
    'LIL': 'darkred',
    'UNKNOWN': 'gray'
}

# Initialize map
m = folium.Map(location=[
    cast_locations["Latitude"].mean(),
    cast_locations["Longitude"].mean()
], zoom_start=13)

# Plot each point with area-based color
for _, row in cast_locations.iterrows():
    popup_text = f"File: {row['File Name']}<br>Lat: {row['Latitude']}<br>Lon: {row['Longitude']}<br>Area: {row['Area']}"
    folium.CircleMarker(
        location=[row["Latitude"], row["Longitude"]],
        radius=5,
        popup=folium.Popup(popup_text, max_width=250),
        color=area_colors.get(row["Area"], "gray"),
        fill=True,
        fill_color=area_colors.get(row["Area"], "gray")
    ).add_to(m)

# Save or display map
m.save("stagecoach_casts_by_area.html")
m


In [40]:
import pandas as pd
from shapely.geometry import Point, Polygon
import folium

# ---- Define Polygons ----
polygon_coords = {
    'DOC': [(40.28429, -106.85365), (40.28360, -106.85341), (40.28240, -106.85488), (40.28356, -106.85524)],
    'HAR': [(40.28424, -106.85348), (40.28565, -106.85185), (40.28797, -106.85680), (40.28793, -106.85836)],
    'MOR': [(40.28003, -106.84560), (40.28021, -106.84274), (40.27252, -106.83760), (40.27183, -106.84056)],
    'GLZ': [(40.27291, -106.84515), (40.27610, -106.85043), (40.27509, -106.85397), (40.27129, -106.84970)],
    'KEY': [(40.27705, -106.86361), (40.27700, -106.86697), (40.28124, -106.86839), (40.28180, -106.86597)],
    'LIL': [(40.27265, -106.86163), (40.27315, -106.85961), (40.27114, -106.85906), (40.27115, -106.86068)],
    'ASP': [(40.272286, -106.876870),(40.272286, -106.876599),(40.272016, -106.876599),(40.272016, -106.876870)]
}

# Shapely Polygons for lookup
polygons = {name: Polygon(coords) for name, coords in polygon_coords.items()}

# ---- Classify Points by Area ----
def get_area(lat, lon):
    point = Point(lat, lon)
    for name, poly in polygons.items():
        if poly.contains(point):
            return name
    return 'MID'

# Assign area
cast_locations['Area'] = cast_locations.apply(lambda row: get_area(row['Latitude'], row['Longitude']), axis=1)

# ---- Color Map for Areas ----
area_colors = {
    'DOC': 'red',
    'HAR': 'green',
    'MOR': 'purple',
    'GLZ': 'orange',
    'KEY': 'darkblue',
    'LIL': 'darkred',
    'ASP': 'pink',
    'MID': 'gray'
}

# ---- Initialize Folium Map ----
m = folium.Map(location=[
    cast_locations["Latitude"].mean(),
    cast_locations["Longitude"].mean()
], zoom_start=13)

# ---- Plot Polygons ----
#for name, coords in polygon_coords.items():
 #   folium.Polygon(
  #      locations=coords,
   #     color=area_colors[name],
    #    fill=True,
     #   fill_opacity=0.15,
      #  popup=f"{name} Area"
   # ).add_to(m)

# ---- Plot Cast Points ----
for _, row in cast_locations.iterrows():
    popup_text = f"File: {row['File Name']}<br>Lat: {row['Latitude']}<br>Lon: {row['Longitude']}<br>Area: {row['Area']}"
    folium.CircleMarker(
        location=[row["Latitude"], row["Longitude"]],
        radius=5,
        popup=folium.Popup(popup_text, max_width=250),
        color=area_colors.get(row["Area"], "gray"),
        fill=True,
        fill_color=area_colors.get(row["Area"], "gray"),
        fill_opacity=0.9
    ).add_to(m)

# ---- Save & Display Map ----
m.save("stagecoach_casts_by_area_with_polygons.html")
m
